In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv("training.csv")

In [4]:
df

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
...,...,...
17995,im having ssa examination tomorrow in the morn...,0
17996,i constantly worry about their fight against n...,1
17997,i feel its important to share this info for th...,1
17998,i truly feel that if you are passionate enough...,1


In [5]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df["text"], df["label"], test_size=0.2)

In [6]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)

In [7]:
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [8]:
maxlen = 50
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')

In [9]:
num_classes = 6
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=50, input_length=maxlen),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=num_classes, activation='softmax')
])

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(train_padded, train_labels, epochs=10, batch_size=32, validation_data=(test_padded, test_labels))

Epoch 1/10
450/450 [==============================] - 12s 19ms/step - loss: 1.4907 - accuracy: 0.3592 - val_loss: 1.2597 - val_accuracy: 0.5581
Epoch 2/10
450/450 [==============================] - 8s 18ms/step - loss: 0.9823 - accuracy: 0.6351 - val_loss: 0.7691 - val_accuracy: 0.6986
Epoch 3/10
450/450 [==============================] - 8s 18ms/step - loss: 0.7543 - accuracy: 0.7086 - val_loss: 0.7652 - val_accuracy: 0.7058
Epoch 4/10
450/450 [==============================] - 8s 17ms/step - loss: 0.6534 - accuracy: 0.7376 - val_loss: 0.6405 - val_accuracy: 0.7439
Epoch 5/10
450/450 [==============================] - 6s 14ms/step - loss: 0.5327 - accuracy: 0.7832 - val_loss: 0.5358 - val_accuracy: 0.7997
Epoch 6/10
450/450 [==============================] - 7s 16ms/step - loss: 0.4210 - accuracy: 0.8558 - val_loss: 0.4629 - val_accuracy: 0.8603
Epoch 7/10
450/450 [==============================] - 7s 17ms/step - loss: 0.3454 - accuracy: 0.8903 - val_loss: 0.4243 - val_accuracy: 0.873

In [13]:
test_loss, test_accuracy = model.evaluate(test_padded, test_labels)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

test_predictions = model.predict(test_padded)
test_pred_labels = np.argmax(test_predictions, axis=1)
test_true_labels = np.argmax(test_labels, axis=1)
print(classification_report(test_true_labels, test_pred_labels))

113/113 [==============================] - 1s 6ms/step - loss: 0.4171 - accuracy: 0.8797
Test loss: 0.41710972785949707
Test accuracy: 0.8797222375869751
113/113 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1063
           1       0.90      0.92      0.91      1206
           2       0.83      0.77      0.80       276
           3       0.95      0.85      0.90       479
           4       0.74      0.81      0.78       452
           5       0.66      0.56      0.60       124

    accuracy                           0.88      3600
   macro avg       0.83      0.81      0.82      3600
weighted avg       0.88      0.88      0.88      3600

